## Importing the data

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(r'C:\Users\hp world\Downloads\mavoix_ml_sample_Dataset.xlsx')
df.head()

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
0,ML0001,Bangalore,0,2,0,2,0,2,3,2,...,0,0,R Programming,Bachelor of Science (B.Sc),Mathematics,2017,NaN,NaN,NaN,NaN
1,ML0002,Bangalore,2,0,0,2,2,2,2,2,...,0,0,"Data Science, Machine Learning, Neural Network...",Bachelor of Technology (B.Tech),Computer Science & Engineering,2019,NaN,85.50/100,92.20/92.20,96.60/96.60
2,ML0003,Bangalore,3,0,1,2,2,2,0,2,...,0,0,"Algorithms, Data Structures, Python, C Program...",Master of Science (M.Sc),Computer Science,2018,7.91/10,70.00/100,64.83/64.83,77.60/77.60
3,ML0004,Bangalore,2,0,2,1,0,2,0,0,...,0,0,"CSS, Deep Learning, Embedded Systems, HTML, Ma...",Bachelor of Engineering (B.E),Electronics and Communication,2019,NaN,6.86/10,76.00/76.00,84.32/84.32
4,ML0005,Bangalore,2,0,0,2,0,2,1,1,...,2,2,"HTML, OpenCV, Python, SQL, C++ Programming, CS...",Bachelor of Technology (B.Tech),Production Engineering,2018,NaN,6.38/10,65.20/65.20,68.80/68.80


In the following section I segregated the skills required for **Data Science & Web Developement** using the domain knowledge and created 2 new columns named **Data Science** and **Web Developement** which contains the avaeraged value of the skill score for both the domains.


Next as I have already taken into account the skills' score by averaging them I dropped the respective columns thus reducing the dimensionality

In [3]:
df['Data Science'] = df['Python (out of 3)'] + df['R Programming (out of 3)']+ df['Deep Learning (out of 3)']+ df['MySQL (out of 3)']+df['Bootstrap (out of 3)']+ df['MongoDB (out of 3)']
df['Data Science'] = df['Data Science']/18
df['Web Developement'] = df['PHP (out of 3)'] + df['HTML (out of 3)']+ df['CSS (out of 3)']+ df['JavaScript (out of 3)']+df['AJAX (out of 3)']+ df['Node.js (out of 3)']+df['ReactJS (out of 3)']
df['Web Developement'] = df['Web Developement']/21

In [6]:
df2 = df.drop(['Current City', 'Python (out of 3)',
       'R Programming (out of 3)', 'Deep Learning (out of 3)',
       'PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)',
       'CSS (out of 3)', 'JavaScript (out of 3)', 'Unnamed: 10',
       'AJAX (out of 3)', 'Bootstrap (out of 3)', 'MongoDB (out of 3)',
       'Node.js (out of 3)', 'ReactJS (out of 3)', 'Other skills', 'Stream'], axis = 1)

### Dropping missing values in the degree column

In [7]:
df2 = df2.dropna(subset = ['Degree'],axis = 0)
df2.shape

(380, 9)

Next as there are a lot of degrees in diffferent fields I tried to simplify it by classifying the degrees broadly into  categories namely : **Bachelors, Masters & Interated** using the following piece of code

In [8]:
df2['Degree'] = df2['Degree'].replace('^B.*','Bachelors',regex = True)
df2['Degree'] = df2['Degree'].replace('^I.*','Integrated',regex = True)
df2['Degree'] = df2['Degree'].replace('^M.*','Masters',regex = True)
df2['Degree'] = df2['Degree'].replace('^P.*','Masters',regex = True)

In [9]:
df2.isnull().sum()

Application_ID                  0
Degree                          0
Current Year Of Graduation      0
Performance_PG                280
Performance_UG                 54
Performance_12                157
Performance_10                171
Data Science                    0
Web Developement                0
dtype: int64

Next I want to drop all those rows in which there are no academic details 

In [10]:
criteria_1 = df2['Performance_PG'].isnull()
criteria_2 = df2['Performance_UG'].isnull() 
criteria_3 = df2['Performance_12'].isnull()
criteria_4 = df2['Performance_10'].isnull()
criteria_all = criteria_1 & criteria_2 & criteria_3 & criteria_4
df3 = df2[criteria_all]
df4 = pd.concat([df2,df3]).drop_duplicates(keep = False)
df4.head()

,Application_ID,Degree,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Data Science,Web Developement
1,ML0002,Bachelors,2019,NaN,85.50/100,92.20/92.20,96.60/96.60,0.222222,0.380952
2,ML0003,Masters,2018,7.91/10,70.00/100,64.83/64.83,77.60/77.60,0.333333,0.285714
3,ML0004,Bachelors,2019,NaN,6.86/10,76.00/76.00,84.32/84.32,0.222222,0.142857
4,ML0005,Bachelors,2018,NaN,6.38/10,65.20/65.20,68.80/68.80,0.222222,0.476190
7,ML0008,Masters,2019,3.61/4,2.68/4,NaN,NaN,0.222222,0.095238


Now if we check the columns containing missing values we will see that those are the columns which contains the academic performance of the candidates, so in the following section I replaced the null values with **00.00/000** in the format given in the data.

As the academic performance columns were of **object** datatype so I further had to use the *.replace()* function along with the required **regex** to convert them into **float** and  finally we have a dataframe where the academic performance scores are numerical figures

In [11]:
df4.fillna('00.00/000',inplace = True)

In [12]:
df4['Performance_PG'] = df4['Performance_PG'].replace('\/\d*', '', regex = True).astype(float)
df4['Performance_UG'] = df4['Performance_UG'].replace('\/\d*', '', regex = True).astype(float)
df4['Performance_12'] = df4['Performance_12'].replace('\/[\d\.]+', '', regex = True).astype(float)
df4['Performance_10'] = df4['Performance_10'].replace('\/[\d\.]+', '', regex = True).astype(float)
df4.head()

,Application_ID,Degree,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10,Data Science,Web Developement
1,ML0002,Bachelors,2019,0.00,85.50,92.20,96.60,0.222222,0.380952
2,ML0003,Masters,2018,7.91,70.00,64.83,77.60,0.333333,0.285714
3,ML0004,Bachelors,2019,0.00,6.86,76.00,84.32,0.222222,0.142857
4,ML0005,Bachelors,2018,0.00,6.38,65.20,68.80,0.222222,0.476190
7,ML0008,Masters,2019,3.61,2.68,0.00,0.00,0.222222,0.095238


It can be seen that  in the academic performance columns like **Performance_pg, Performance_UG** the format of marks were different, like for few candidates it was a CGPA out of 10 whereas for few the scoring was in GPA system out of 4 where as others had percentage out of 100:

SO I tried to normalize this by multiplying 10 with the ones whose score was out off 10 and 25 with the ones whose scores were out of 4 in the following section

In [13]:
z = []
for i in df4['Performance_PG']:
    if i <= 10 and i > 4:
        i = i * 10
    elif i <= 4 and i >= 0:
        i = i * 25
    z.append(i)      
df4['Performance_PG'] = z

In [14]:
x = []
for i in df4['Performance_UG']:
    if i <= 10 and i > 4:
        i = i * 10
    elif i <= 4 and i >= 0:
        i = i * 25
    x.append(i)    
df4['Performance_UG'] = x

In [15]:
w = []
for i in df4['Performance_12']:
    if i <= 10 and i > 4:
        i = i * 10
    elif i <= 4 and i >= 0:
        i = i * 25
    w.append(i)    
df4['Performance_12'] = w

In [16]:
y = []
for i in df4['Performance_10']:
    if i <= 10 and i > 4:
        i = i * 10
    elif i <= 4 and i >= 0:
        i = i * 25
    y.append(i)    
df4['Performance_10'] = y

As for many of the candidates the few scores from PG were missing as they just had Bachelors degree and similarly for other reasons as well, so I tried to normalize this factor by giving seperate weightage to different degree scores, like as for any job Bachelor Degree is very important I decided to take 60% out if the score of UG, as PG is a plus, I took 20% score from it and 15% and 5% from 12 and 10 respectively.

Then I created aa new column named **Academic_Score** and and added the values in the below mentioned format and dropped the columns containing the performance scores in different degrees as these are already accounted for thus further reducing the dimentionality

In [17]:
df4['Academic_Score'] = (df4['Performance_PG']*0.20) + (df4['Performance_UG']*0.6) + (df4['Performance_12']*0.15) + (df4['Performance_10']*0.05)

In [18]:
df5 = df4.drop(['Performance_PG','Performance_UG', 'Performance_12', 'Performance_10'], axis = 1)
df5.head()

,Application_ID,Degree,Current Year Of Graduation,Data Science,Web Developement,Academic_Score
1,ML0002,Bachelors,2019,0.222222,0.380952,69.9600
2,ML0003,Masters,2018,0.333333,0.285714,71.4245
3,ML0004,Bachelors,2019,0.222222,0.142857,56.7760
4,ML0005,Bachelors,2018,0.222222,0.476190,51.5000
7,ML0008,Masters,2019,0.222222,0.095238,58.2500


In [19]:
df5.describe()

,Current Year Of Graduation,Data Science,Web Developement,Academic_Score
count,348.000000,348.000000,348.000000,348.000000
mean,2019.002874,0.170817,0.194308,53.721286
std,1.898731,0.126393,0.162236,14.382440
min,2008.000000,0.000000,0.000000,12.400000
25%,2019.000000,0.111111,0.035714,45.840000
50%,2019.000000,0.166667,0.190476,55.585000
75%,2020.000000,0.222222,0.333333,63.116250
max,2023.000000,0.722222,0.571429,88.567500


## Model Building

I used the **Unsupervised Learning Algorithm, KMeans** with a cluster value of 2 in order to group the data

In [20]:
from sklearn.cluster import KMeans

In [21]:
km = KMeans(n_clusters = 2)
predictions = km.fit_predict(df5.iloc[:,3:6])
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,

In [22]:
df5['labels'] = predictions

In [23]:
label_1 = df5.loc[df5['labels']==1]
label_0 = df5.loc[df5['labels']==0]

In [24]:
label_1.describe()

,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels
count,128.000000,128.000000,128.000000,128.000000,128.0
mean,2018.585938,0.164931,0.183780,39.180297,1.0
std,2.207815,0.120741,0.152673,11.200281,0.0
min,2008.000000,0.000000,0.000000,12.400000,1.0
25%,2018.000000,0.055556,0.000000,35.883375,1.0
50%,2019.000000,0.166667,0.190476,43.440000,1.0
75%,2020.000000,0.222222,0.285714,46.825000,1.0
max,2023.000000,0.500000,0.523810,50.500000,1.0


In [25]:
label_0.describe()

,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels
count,220.000000,220.000000,220.000000,220.000000,220.0
mean,2019.245455,0.174242,0.200433,62.181498,0.0
std,1.650717,0.129715,0.167585,7.715238,0.0
min,2012.000000,0.000000,0.000000,50.800000,0.0
25%,2019.000000,0.111111,0.047619,56.347250,0.0
50%,2020.000000,0.111111,0.190476,60.565000,0.0
75%,2020.000000,0.222222,0.333333,67.545000,0.0
max,2023.000000,0.722222,0.571429,88.567500,0.0


By analyzing the above dataframes I could say that the dataframe **label_1** contains the details of the candidates whose academic performance is below par where as the dataframe **label_0** contains the details of the candidates whose academic performance is good and also their scores in **Data Science & Web Developement** is better than the previous 


So I decided to take the **label_0** dataframe and apply **KMeans** algorithm on it again this time passing only the **Data Science & Web Developement** column

In [26]:
label_0.head()

,Application_ID,Degree,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels
1,ML0002,Bachelors,2019,0.222222,0.380952,69.9600,0
2,ML0003,Masters,2018,0.333333,0.285714,71.4245,0
3,ML0004,Bachelors,2019,0.222222,0.142857,56.7760,0
4,ML0005,Bachelors,2018,0.222222,0.476190,51.5000,0
7,ML0008,Masters,2019,0.222222,0.095238,58.2500,0


In [29]:
eligibles = km.fit_predict(label_0.iloc[:,3:5])
eligibles

array([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0])

In [30]:
label_0['profile'] = eligibles
label_0.head()

C:\Users\hp world\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Application_ID,Degree,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels,profile
1,ML0002,Bachelors,2019,0.222222,0.380952,69.9600,0,0
2,ML0003,Masters,2018,0.333333,0.285714,71.4245,0,0
3,ML0004,Bachelors,2019,0.222222,0.142857,56.7760,0,1
4,ML0005,Bachelors,2018,0.222222,0.476190,51.5000,0,0
7,ML0008,Masters,2019,0.222222,0.095238,58.2500,0,1


## Cluster Analysis for the clusters formed on the label_0 dataframe

In [33]:
new_1 = label_0.loc[label_0['profile']==1]
new_2 = label_0.loc[label_0['profile']==0]

In [34]:
new_1.describe()

,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels,profile
count,128.000000,128.000000,128.000000,128.000000,128.0,128.0
mean,2019.132812,0.135851,0.079241,62.109047,0.0,1.0
std,1.941552,0.083895,0.078662,8.156852,0.0,0.0
min,2012.000000,0.000000,0.000000,51.000000,0.0,1.0
25%,2019.000000,0.055556,0.000000,55.304250,0.0,1.0
50%,2020.000000,0.111111,0.095238,60.270000,0.0,1.0
75%,2020.000000,0.222222,0.142857,67.545000,0.0,1.0
max,2023.000000,0.333333,0.238095,88.567500,0.0,1.0


In [35]:
new_2.describe()

,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels,profile
count,92.000000,92.000000,92.000000,92.000000,92.0,92.0
mean,2019.402174,0.227657,0.369048,62.282299,0.0,0.0
std,1.119849,0.160352,0.097724,7.097823,0.0,0.0
min,2016.000000,0.000000,0.190476,50.800000,0.0,0.0
25%,2019.000000,0.111111,0.285714,56.773125,0.0,0.0
50%,2019.000000,0.194444,0.380952,61.090000,0.0,0.0
75%,2020.000000,0.333333,0.428571,67.347500,0.0,0.0
max,2022.000000,0.722222,0.571429,81.690000,0.0,0.0


Upon analysis from the above tables I found out that the new_2 dataframe has got the details of the candidates whose academic performance is good as well as their mean scores in the **Data Science and Web Developemnt** skills is also good.

Basically it was seen that the overall Data Science Score was around 13% and that of Web Developement was around 16%, but here in this cluster we can see that the average Data Science Score is around 23% and that of Web Developement is almost 37% abd the average academic score is also about 62% which is why is considered the candidates in this cluster to be suitable for further consideration.




In [36]:
new_2.head()

,Application_ID,Degree,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels,profile
1,ML0002,Bachelors,2019,0.222222,0.380952,69.9600,0,0
2,ML0003,Masters,2018,0.333333,0.285714,71.4245,0,0
4,ML0005,Bachelors,2018,0.222222,0.476190,51.5000,0,0
9,ML0010,Masters,2020,0.222222,0.380952,60.8500,0,0
16,ML0017,Masters,2019,0.111111,0.285714,64.5500,0,0


But again we want the creamy layer to be considered so I considered the median value for Data Science score as well as Web Developement and decided to choose the candidates who are above the median score from the new_2 dataframe and they already have a good academic record

In [37]:
Data_Science = pd.DataFrame()

In [42]:
Data_Science = new_2[new_2['Data Science']>0.20]
Data_Science.head()

,Application_ID,Degree,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels,profile
1,ML0002,Bachelors,2019,0.222222,0.380952,69.9600,0,0
2,ML0003,Masters,2018,0.333333,0.285714,71.4245,0,0
4,ML0005,Bachelors,2018,0.222222,0.476190,51.5000,0,0
9,ML0010,Masters,2020,0.222222,0.380952,60.8500,0,0
19,ML0020,Masters,2019,0.222222,0.428571,54.6500,0,0


In [43]:
Web_Developement = new_2[new_2['Web Developement']>0.38]
Web_Developement.head()

,Application_ID,Degree,Current Year Of Graduation,Data Science,Web Developement,Academic_Score,labels,profile
1,ML0002,Bachelors,2019,0.222222,0.380952,69.960,0,0
4,ML0005,Bachelors,2018,0.222222,0.476190,51.500,0,0
9,ML0010,Masters,2020,0.222222,0.380952,60.850,0,0
19,ML0020,Masters,2019,0.222222,0.428571,54.650,0,0
38,ML0039,Masters,2019,0.333333,0.523810,65.145,0,0


### Taking out the application number which should be considered for Data Science role

In [44]:
Data_Scientist = Data_Science['Application_ID']
Data_Scientist

1      ML0002
2      ML0003
4      ML0005
9      ML0010
19     ML0020
33     ML0034
34     ML0035
36     ML0037
38     ML0039
54     ML0055
66     ML0067
71     ML0072
80     ML0081
93     ML0094
112    ML0113
124    ML0125
134    ML0135
135    ML0136
139    ML0140
154    ML0155
156    ML0157
171    ML0172
175    ML0176
180    ML0181
202    ML0203
205    ML0206
206    ML0207
220    ML0221
222    ML0223
224    ML0225
232    ML0233
240    ML0241
249    ML0250
254    ML0255
258    ML0259
279    ML0280
285    ML0286
291    ML0292
302    ML0303
308    ML0309
313    ML0314
344    ML0345
359    ML0360
374    ML0375
378    ML0379
391    ML0392
Name: Application_ID, dtype: object

### Taking out the application number which should be considered for Data Science role

In [46]:
Web_developer = Web_Developement['Application_ID']
Web_developer

1      ML0002
4      ML0005
9      ML0010
19     ML0020
38     ML0039
41     ML0042
51     ML0052
66     ML0067
71     ML0072
98     ML0099
112    ML0113
124    ML0125
139    ML0140
157    ML0158
171    ML0172
181    ML0182
185    ML0186
195    ML0196
205    ML0206
206    ML0207
209    ML0210
220    ML0221
221    ML0222
224    ML0225
229    ML0230
240    ML0241
257    ML0258
258    ML0259
261    ML0262
271    ML0272
285    ML0286
286    ML0287
291    ML0292
293    ML0294
308    ML0309
309    ML0310
314    ML0315
316    ML0317
318    ML0319
327    ML0328
332    ML0333
340    ML0341
344    ML0345
359    ML0360
377    ML0378
378    ML0379
381    ML0382
391    ML0392
Name: Application_ID, dtype: object